In [1]:
from bidict import bidict
from collections import defaultdict
import itertools
import os
import pandas as pd
import pickle
import lzma

from conllu import parse, parse_incr
import numpy as np
from cp_orth import orth_als
import sktensor

from decomp_pmi import VerbTensor

import logging
logging.basicConfig(level=logging.DEBUG, format='%(levelname)-8s [%(lineno)d] %(message)s')

# DepCC: salience

In [2]:
svo_count = pd.read_csv('/mnt/store/home/makrai/project/verb-tensor/just_svo/depCC-.tsv', sep='\t')

In [3]:
svo_count.head()#sort_values('freq', ascending=False).head()

,nsubj,ROOT,dobj,freq
0,we,support,browser,4706
1,we,reserve,right,2996
2,I,have,idea,2173
3,WebMD,disclaim,warranty,2017
4,I,love,it,1839


In [4]:
svo_count = VerbTensor().append_pmi()

INFO     [34] Readding freq counts from /mnt/store/home/makrai/project/verb-tensor/just_svo/dataframe/depCC/freq0.tsv
INFO     [36] Computing marginals (1/2)..
DEBUG    [44] ('nsubj', 'ROOT')
DEBUG    [44] ('nsubj', 'dobj')
DEBUG    [44] ('ROOT', 'dobj')
INFO     [47] Computing Dice..
INFO     [56] Computing PMI variants..
DEBUG    [67] 0   -7.004859
1   -7.656322
2   -8.119672
3   -8.227149
4   -8.360438
Name: pmi, dtype: float64
DEBUG    [72] 0     9.413332
1     9.750295
2     3.086194
3    16.158361
4     4.493513
Name: pmi, dtype: float64
DEBUG    [75] 0     9.413332
1     9.750295
2     3.086194
3    16.158361
4     4.493513
Name: pmi, dtype: float64
DEBUG    [77] 0    38.410288
1    38.410288
2    38.410288
3    38.410288
4    38.410288
Name: pmi, dtype: float64
INFO     [80] Computing salience..
INFO     [83] Saving to /mnt/store/home/makrai/project/verb-tensor/just_svo/dataframe/depCC/assoc0.{}..


In [7]:
svo_count['0'] = 0

In [9]:
svo_count[['pmi', '0']].max(axis=1)

0         38.410288
1         38.410288
2         38.410288
3         38.410288
4         38.410288
5         38.410288
6         38.410288
7         38.410288
8         38.410288
9         38.410288
10        38.410288
11        38.410288
12        38.410288
13        38.410288
14        38.410288
15        38.410288
16        38.410288
17        38.410288
18        38.410288
19        38.410288
20        38.410288
21        38.410288
22        38.410288
23        38.410288
24        38.410288
25        38.410288
26        38.410288
27        38.410288
28        38.410288
29        38.410288
            ...    
335093    38.410288
335094    38.410288
335095    38.410288
335096    38.410288
335097    38.410288
335098    38.410288
335099    38.410288
335100    38.410288
335101    38.410288
335102    38.410288
335103    38.410288
335104    38.410288
335105    38.410288
335106    38.410288
335107    38.410288
335108    38.410288
335109    38.410288
335110    38.410288
335111    38.410288


In [45]:
svo_count.describe(percentiles=[]).loc['std']

freq                      1.378200e+01
freq_nsubj                6.439916e+00
freq_ROOT                 4.020314e+00
freq_dobj                 3.453074e+00
freq_('nsubj', 'ROOT')    4.632857e+00
freq_('nsubj', 'dobj')    2.449073e+00
freq_('ROOT', 'dobj')     2.377650e+00
log_freq                  7.614164e-01
log_dice                  3.299808e+00
pmi                       2.131632e-14
iact_info                 4.973807e-14
salience                  2.910449e+01
iact_sali                 3.878152e+01
Name: std, dtype: float64

In [29]:
svo_count[svo_count.freq>1].sort_values('pmi', ascending=False).head()

,nsubj,ROOT,dobj,freq,freq_nsubj,freq_ROOT,freq_dobj,"freq_('nsubj', 'ROOT')","freq_('nsubj', 'dobj')","freq_('ROOT', 'dobj')",log_freq,log_dice,pmi,iact_info,salience,iact_sali
0,we,support,browser,3450,-2.931951,-6.442409,-6.924641,-6.861104,-6.939612,-6.937941,-6.940031,-2.625463,37.384822,49.538509,-259.451809,-343.798762
25867,they,deceive,we,2,-4.431198,-15.885056,-10.103697,-17.692411,-12.991971,-17.107449,-17.692411,-11.704768,37.384822,49.538509,-661.427649,-876.455662
25869,this,include,testimony,2,-4.969924,-4.832779,-12.885056,-7.176711,-16.692411,-17.692411,-17.692411,-12.210570,37.384822,49.538509,-661.427649,-876.455662
25870,CENTCOM,report,soldier,2,-17.692411,-9.283020,-13.300094,-17.692411,-17.692411,-17.692411,-17.692411,-6.914883,37.384822,49.538509,-661.427649,-876.455662
25871,I,use,weapon,2,-2.078147,-6.301705,-12.444484,-7.781019,-15.885056,-16.370483,-17.692411,-14.105568,37.384822,49.538509,-661.427649,-876.455662


In [27]:
svo_count[svo_count.freq>1].sort_values('iact_info', ascending=False).head()

,nsubj,ROOT,dobj,freq,freq_nsubj,freq_ROOT,freq_dobj,"freq_('nsubj', 'ROOT')","freq_('nsubj', 'dobj')","freq_('ROOT', 'dobj')",log_freq,log_dice,pmi,iact_info,salience,iact_sali
0,we,support,browser,3450,-2.931951,-6.442409,-6.924641,-6.861104,-6.939612,-6.937941,-6.940031,-2.625463,37.384822,49.538509,-259.451809,-343.798762
25867,they,deceive,we,2,-4.431198,-15.885056,-10.103697,-17.692411,-12.991971,-17.107449,-17.692411,-11.704768,37.384822,49.538509,-661.427649,-876.455662
25869,this,include,testimony,2,-4.969924,-4.832779,-12.885056,-7.176711,-16.692411,-17.692411,-17.692411,-12.210570,37.384822,49.538509,-661.427649,-876.455662
25870,CENTCOM,report,soldier,2,-17.692411,-9.283020,-13.300094,-17.692411,-17.692411,-17.692411,-17.692411,-6.914883,37.384822,49.538509,-661.427649,-876.455662
25871,I,use,weapon,2,-2.078147,-6.301705,-12.444484,-7.781019,-15.885056,-16.370483,-17.692411,-14.105568,37.384822,49.538509,-661.427649,-876.455662


In [34]:
svo_count[svo_count.freq>20].sort_values('log_dice', ascending=False).head()

,nsubj,ROOT,dobj,freq,freq_nsubj,freq_ROOT,freq_dobj,"freq_('nsubj', 'ROOT')","freq_('nsubj', 'dobj')","freq_('ROOT', 'dobj')",log_freq,log_dice,pmi,iact_info,salience,iact_sali
307,ISPE,s.r.l.,Palazzolo,61,-12.761674,-12.761674,-12.761674,-12.761674,-12.761674,-12.761674,-12.761674,0.000000,37.384822,49.538509,-477.092909,-632.194288
34,Donec,id,turpis,256,-10.670043,-10.599654,-10.686787,-10.692411,-10.692411,-10.692411,-10.692411,-0.040746,37.384822,49.538509,-399.733892,-529.686102
695,bayou,goat,Mounts,34,-13.563128,-13.563128,-13.563128,-13.604948,-13.604948,-13.604948,-13.604948,-0.041820,37.384822,49.538509,-508.618577,-673.968849
82,@param,cred,credential,153,-11.241200,-11.407009,-11.397790,-11.407009,-11.425625,-11.435023,-11.435023,-0.088387,37.384822,49.538509,-427.496316,-566.474001
2,WebMD,disclaim,warranty,1473,-8.068530,-8.119711,-8.038670,-8.167869,-8.167869,-8.156164,-8.167869,-0.092620,37.384822,49.538509,-305.354349,-404.624071


In [35]:
svo_count.sort_values('salience', ascending=False).head()

,nsubj,ROOT,dobj,freq,freq_nsubj,freq_ROOT,freq_dobj,"freq_('nsubj', 'ROOT')","freq_('nsubj', 'dobj')","freq_('ROOT', 'dobj')",log_freq,log_dice,pmi,iact_info,salience,iact_sali
0,we,support,browser,3450,-2.931951,-6.442409,-6.924641,-6.861104,-6.939612,-6.937941,-6.940031,-2.625463,37.384822,49.538509,-259.451809,-343.798762
1,we,reserve,right,2187,-2.931951,-7.451620,-6.924227,-7.591749,-7.514992,-7.493352,-7.597674,-3.226674,37.384822,49.538509,-284.037681,-376.377422
2,WebMD,disclaim,warranty,1473,-8.068530,-8.119711,-8.038670,-8.167869,-8.167869,-8.156164,-8.167869,-0.092620,37.384822,49.538509,-305.354349,-404.624071
3,I,have,idea,1473,-2.078147,-2.186429,-6.963640,-3.884905,-7.757983,-7.508156,-8.167869,-5.476735,37.384822,49.538509,-305.354349,-404.624071
4,I,love,it,1276,-2.078147,-4.791639,-5.959608,-5.149742,-6.880234,-7.973022,-8.374999,-4.999140,37.384822,49.538509,-313.097834,-414.884938


In [36]:
svo_count[svo_count.freq>=0].sort_values('iact_sali', ascending=False).head()

,nsubj,ROOT,dobj,freq,freq_nsubj,freq_ROOT,freq_dobj,"freq_('nsubj', 'ROOT')","freq_('nsubj', 'dobj')","freq_('ROOT', 'dobj')",log_freq,log_dice,pmi,iact_info,salience,iact_sali
0,we,support,browser,3450,-2.931951,-6.442409,-6.924641,-6.861104,-6.939612,-6.937941,-6.940031,-2.625463,37.384822,49.538509,-259.451809,-343.798762
1,we,reserve,right,2187,-2.931951,-7.451620,-6.924227,-7.591749,-7.514992,-7.493352,-7.597674,-3.226674,37.384822,49.538509,-284.037681,-376.377422
2,WebMD,disclaim,warranty,1473,-8.068530,-8.119711,-8.038670,-8.167869,-8.167869,-8.156164,-8.167869,-0.092620,37.384822,49.538509,-305.354349,-404.624071
3,I,have,idea,1473,-2.078147,-2.186429,-6.963640,-3.884905,-7.757983,-7.508156,-8.167869,-5.476735,37.384822,49.538509,-305.354349,-404.624071
4,I,love,it,1276,-2.078147,-4.791639,-5.959608,-5.149742,-6.880234,-7.973022,-8.374999,-4.999140,37.384822,49.538509,-313.097834,-414.884938


# Get UMBC

In [ ]:
def get_umbc_dict():
    umbc_dir = '/mnt/store/home/makrai/data/language/english/corp/umbc_WebBase/English/'
    freq = defaultdict(int)
    for filen in os.listdir(umbc_dir):
        logging.info(filen)
        for i, sentence in  enumerate(parse_incr(lzma.open(os.path.join(umbc_dir, filen), mode='rt',
                                                           encoding="utf-8"))):
            if not i % 100000:
                logging.debug(i)
            root = sentence.to_tree()
            subj, obj = '', ''
            for child in root.children:
                if 'subj' in child.token['deprel']:
                    if subj:
                        #logging.warn('subj: {}'.format((subj, child.token['lemma'], sentence)))
                        continue
                    subj = child.token['lemma']
                elif child.token['deprel'] == 'obj':
                    if obj:
                        #logging.warn('obj: {}'.format((obj, child.token['lemma'], sentence)))
                        continue            
                    obj = child.token['lemma']
            #if bool(obj) and bool(subj):
            freq[(subj, root.token['lemma'], obj)] += 1
        #pickle.dump(freq, open('/mnt/store/home/makrai/project/verb-tensor/umbc_freq.pkl', mode='wb'))
    return freq

In [ ]:
def get_umbc_df():
    freq = pickle.load(open('/mnt/store/home/makrai/project/verb-tensor/umbc_freq.pkl', mode='rb'))
    freq_df = pd.DataFrame.from_records(list(freq.items()), columns=['svo', 'freq'])
    freq_df[['subj', 'verb', 'obj']] = pd.DataFrame(freq_df.svo.tolist(), index=freq_df.index)                                                                                                                       
    del freq_df['svo']
    return freq_df

# Mazsola DB

In [ ]:
mazsola = pickle.load(open(
    '/mnt/permanent/Language/Hungarian/Dic/sass15-535k-igei-szerkezet/mazsola_adatbazis.pkl',
    mode='rb'))

In [ ]:
mazsola_df =pd.read_csv(
    '/mnt/permanent/Language/Hungarian/Dic/sass15-535k-igei-szerkezet/mazsola_adatbazis_svo_freq.tsv', sep='\t',
    keep_default_na=False)

In [ ]:
mazsola_df, log_total = append_pmi(mazsola_df, compute_freq=False)

In [ ]:
mazsola_df.sort_values('iact_info').head()

# Top triples

In [ ]:
svo_count.sort_values('freq', ascending=False).head()

In [ ]:
svo_count[svo_count.freq>100].sort_values('pmi', ascending=False).head()

In [ ]:
svo_count[(svo_count.freq>100) & (svo_count.ACC != 'NULL')].sort_values('iact_info', ascending=False).head()

In [ ]:
svo_count[(svo_count.ACC != 'NULL')].sort_values('salience', ascending=False).head()

In [ ]:
svo_count.sort_values('iact_sali', ascending=False).head()

## UMBC

In [ ]:
svo_count, log_total = append_pmi(freq_df, modes=['subj', 'verb', 'obj'], compute_freq=False)#, debug_index=2234759)

In [ ]:
svo_count[(svo_count.subj != '') & (svo_count.obj != '')].sort_values('freq', ascending=False).head()

In [ ]:
svo_count[svo_count.freq>100].sort_values('pmi', ascending=False).head()

In [ ]:
svo_count[(svo_count.freq>100) & (svo_count.subj != '') & (svo_count.obj != '')].sort_values(
    'iact_info', ascending=True).head()

In [ ]:
svo_count[(svo_count.subj != '') & (svo_count.obj != '')].sort_values('salience', ascending=False).head()

In [ ]:
svo_count[(svo_count.freq>100)].sort_values('dice', ascending=False).head()